In [18]:
# DIC CSV → per-frame summaries (tailored to your headers)


from pathlib import Path
import re, os
import numpy as np
import pandas as pd

try:
    from tqdm import tqdm
except Exception:
    tqdm = lambda x, **k: x  # no-op if tqdm is absent

# -------- PATHS --------
DIC_ROOT    = Path("Dissertation/Hari Export")             # folder with all DIC CSVs (recursive)
OUT_DIR     = Path("./outputs"); OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_SUMMARY = OUT_DIR / "dic_summary.csv"

# -------- EXACT COLUMN NAMES (as seen in your CSV) --------
COL_U    = "disp.Horizontal Displacement U [Pixel]"
COL_V    = "disp.Vertical Displacement V [Pixel]"
COL_EXX  = "strain.Strain-local frame: Exx [ ]"
COL_EYY  = "strain.Strain-local frame: Eyy [ ]"
COL_EXY  = "strain.Strain-local frame: Exy [ ]"
COL_VM   = "strain.Von Mises Equivalent Strain [ ]"
COL_FRAMEIDX = "ext.Current Image [#]"   # optional

# -------- HELPERS --------
def frame_id_from_path(p: Path) -> str:
    """Strip .csv and any trailing .tif/.tiff from filename to get a frame_id like '20cm_0071'."""
    name = p.name
    if name.lower().endswith(".csv"):  name = name[:-4]
    name = re.sub(r"\.(tif|tiff)$", "", name, flags=re.IGNORECASE)
    return name

def nanpercentile(a, q):
    a = np.asarray(a)
    if a.size == 0: return np.nan
    a = a[np.isfinite(a)]
    if a.size == 0: return np.nan
    return float(np.percentile(a, q))



In [19]:
def summarise_dic_csv(csv_path: Path):
    """
    Reduce one frame CSV into a few summaries:
      - n_subsets
      - mean_disp_px, p95_disp_px (from U,V)
      - mean_vmises
      - mean_exx, mean_eyy, mean_exy  (optional but cheap)
    Returns a dict (or None if unreadable).
    """
    usecols = [c for c in [COL_U, COL_V, COL_EXX, COL_EYY, COL_EXY, COL_VM, COL_FRAMEIDX] if c is not None]
    try:
        df = pd.read_csv(csv_path, usecols=usecols, dtype=np.float32, engine="c", low_memory=False)
    except Exception as e:
        print(f"[WARN] read_csv failed: {csv_path} -> {e}")
        return None

    out = {
        "frame_id": frame_id_from_path(csv_path),
        "n_subsets": int(len(df)),
        "mean_disp_px": np.nan,
        "p95_disp_px":  np.nan,
        "mean_vmises":  np.nan,
        "mean_exx":     np.nan,
        "mean_eyy":     np.nan,
        "mean_exy":     np.nan,
    }

    # --- Displacement magnitude ---
    if (COL_U in df.columns) and (COL_V in df.columns):
        U = df[COL_U].to_numpy()
        V = df[COL_V].to_numpy()
        disp = np.sqrt(U*U + V*V)
        disp = disp[np.isfinite(disp)]
        if disp.size:
            out["mean_disp_px"] = float(np.nanmean(disp))
            out["p95_disp_px"]  = nanpercentile(disp, 95)

    # --- Von Mises strain ---
    if COL_VM in df.columns:
        vm = df[COL_VM].to_numpy()
        vm = vm[np.isfinite(vm)]
        if vm.size:
            out["mean_vmises"] = float(np.nanmean(vm))

    # --- Strain tensor components (optional means) ---
    for col, key in [(COL_EXX, "mean_exx"), (COL_EYY, "mean_eyy"), (COL_EXY, "mean_exy")]:
        if col in df.columns:
            vals = df[col].to_numpy()
            vals = vals[np.isfinite(vals)]
            if vals.size:
                out[key] = float(np.nanmean(vals))

    return out


In [20]:
# Find all CSVs under Hari_Export (recursively)
# Discover CSVs
csv_files = sorted(DIC_ROOT.rglob("*.csv"))
print(f"Found {len(csv_files)} CSV files under {DIC_ROOT.resolve()}")

# Parallel process files
from concurrent.futures import ProcessPoolExecutor, as_completed
results = []
num_workers = max(1, (os.cpu_count() or 2) - 1)
print(f"Using {num_workers} workers")

with ProcessPoolExecutor(max_workers=num_workers) as ex:
    futs = {ex.submit(summarise_dic_csv, p): p for p in csv_files}
    for fut in tqdm(as_completed(futs), total=len(futs), desc="Summarising"):
        rec = fut.result()
        if rec is not None:
            results.append(rec)

df_dic = pd.DataFrame(results)
if df_dic.empty:
    raise RuntimeError("No summaries produced — check CSV format/paths.")

# Sort and save
df_dic = df_dic.sort_values("frame_id").reset_index(drop=True)
df_dic.to_csv(OUT_SUMMARY, index=False)
print(f"Wrote {len(df_dic)} rows to {OUT_SUMMARY.resolve()}")
display(df_dic.head())



Found 1693 CSV files under /home/ubuntu/Dissertation/Hari Export
Using 3 workers


Summarising: 100%|██████████████████████████| 1693/1693 [01:27<00:00, 19.38it/s]

Wrote 1693 rows to /home/ubuntu/outputs/dic_summary.csv


,frame_id,n_subsets,mean_disp_px,p95_disp_px,mean_vmises,mean_exx,mean_eyy,mean_exy
0,0cm_0000,48884,0.067775,0.081151,0.000245,6.978449e-07,7.720270e-06,0.000009
1,0cm_0001,48884,0.076478,0.106096,0.000202,-1.672498e-05,-5.284778e-06,-0.000020
2,0cm_0002,48884,0.098983,0.121090,0.000226,6.927095e-06,2.830703e-06,0.000009
3,0cm_0003,48884,0.129337,0.163201,0.000220,-5.245639e-06,-2.995358e-05,-0.000018
4,0cm_0004,48884,0.106204,0.157112,0.000240,2.668238e-05,9.203127e-07,-0.000027


## Load DIC + model scores, standardise IDs, and join

In [25]:
# --- CELL 1: load + normalise + join (drop-in replacement) ---
from pathlib import Path
import pandas as pd, numpy as np, re

OUT_DIR = Path("outputs")
FIGS_DIR = OUT_DIR / "figs"; FIGS_DIR.mkdir(parents=True, exist_ok=True)

DIC_CSV   = OUT_DIR / "dic_summary.csv"     # DIC reduction
PBAR_CSV  = OUT_DIR / "pbar_test.csv"       # non-overlap frame scores

THR = 0.882  # your current Youden threshold

df_d = pd.read_csv(DIC_CSV)
df_p = pd.read_csv(PBAR_CSV)

def _norm_id(s):
    s = str(s)
    return re.sub(r"\.(csv|tif|tiff)$", "", s, flags=re.IGNORECASE)

for col in ["frame_id"]:
    if col in df_d: df_d[col] = df_d[col].map(_norm_id)
    if col in df_p: df_p[col] = df_p[col].map(_norm_id)

# map class to string (if present)
if "class" in df_p.columns:
    def _to_name(v):
        try:
            return "haze" if int(v)==1 else "nonhaze"
        except Exception:
            return "haze" if "haze" in str(v).lower() and "non" not in str(v).lower() else "nonhaze"
    df_p["class_name"] = df_p["class"].map(_to_name)
else:
    df_p["class_name"] = "unknown"

dfj = df_p.merge(df_d, on="frame_id", how="inner")
dfj_haze = dfj[dfj["class_name"]=="haze"].copy() if "class_name" in dfj.columns else dfj.copy()

print(f"Joined frames: total={len(dfj)}, haze_subset={len(dfj_haze)}")
dfj_haze.head()


Joined frames: total=249, haze_subset=249


,split,frame_id,class,n_tiles,pbar,class_name,n_subsets,mean_disp_px,p95_disp_px,mean_vmises,mean_exx,mean_eyy,mean_exy
0,test,0cm_0020,haze,4,0.999793,haze,48884,0.044733,0.062678,0.000310,-1.231292e-05,0.000004,0.000002
1,test,0cm_0035,haze,4,0.998599,haze,48884,0.084901,0.099869,0.000340,-2.089861e-07,0.000009,-0.000005
2,test,0cm_0042,haze,4,0.999888,haze,48884,0.055209,0.083793,0.000325,1.721081e-05,0.000012,0.000018
3,test,0cm_0049,haze,4,0.998309,haze,48884,0.054215,0.090791,0.000435,-3.094885e-05,-0.000008,-0.000020
4,test,0cm_0050,haze,4,0.998402,haze,48884,0.100656,0.161727,0.000354,-6.522970e-05,-0.000026,0.000013


## Compute headline stats

In [26]:

import numpy as np

def spearman_rho(x, y):
    try:
        from scipy.stats import spearmanr
        rho, p = spearmanr(x, y, nan_policy='omit')
        return float(rho), float(p)
    except Exception:
        # fallback: Pearson of ranks
        xr = pd.Series(x).rank().to_numpy()
        yr = pd.Series(y).rank().to_numpy()
        rho = np.corrcoef(xr, yr)[0,1]
        return float(rho), np.nan

def coverage_at_T(p, T):
    p = np.asarray(p)
    return float((p >= T).mean()) if p.size else np.nan

# Choose the DIC intensity measures to compare with pbar
y_pbar  = dfj_haze["pbar"].to_numpy()
x_p95   = dfj_haze["p95_disp_px"].to_numpy()     # “worst pockets” of motion
x_vm    = dfj_haze.get("mean_vmises", pd.Series(dtype=float)).to_numpy()  # unitless strain

# Coverage
covT = coverage_at_T(y_pbar, THR)

# Spearman correlations
rho_p95, p_p95 = spearman_rho(x_p95, y_pbar)
rho_vm,  p_vm  = spearman_rho(x_vm,  y_pbar) if len(x_vm)>0 else (np.nan, np.nan)

# Optional: top-k overlap (k = max(1, 10% of set))
N = len(dfj_haze)
k = max(1, int(0.10 * N))
idx_top_pbar = np.argsort(-y_pbar)[:k]
idx_top_p95  = np.argsort(-x_p95)[:k]
top_overlap  = len(set(idx_top_pbar).intersection(set(idx_top_p95)))
top_jaccard  = top_overlap / float(2*k - top_overlap) if k>0 else np.nan

# Save a tiny summary CSV for LaTeX
summary = pd.DataFrame([{
    "n_frames_joined": N,
    "threshold_T": THR,
    "coverage_at_T": covT,
    "spearman_rho_pbar_vs_p95disp": rho_p95,
    "p_value_p95disp": p_p95,
    "spearman_rho_pbar_vs_vmises": rho_vm,
    "p_value_vmises": p_vm,
    "topk_k": k,
    "topk_overlap_count": top_overlap,
    "topk_jaccard": top_jaccard
}])
summary_path = OUT_DIR / "dic_x_model_summary.csv"
summary.to_csv(summary_path, index=False)

print(summary.round(3))
print(f"Saved: {summary_path}")


   n_frames_joined  threshold_T  coverage_at_T  spearman_rho_pbar_vs_p95disp  \
0              249        0.882          0.996                        -0.219   

   p_value_p95disp  spearman_rho_pbar_vs_vmises  p_value_vmises  topk_k  \
0            0.001                        0.293             0.0      24   

   topk_overlap_count  topk_jaccard  
0                   0           0.0  
Saved: outputs/dic_x_model_summary.csv


## Plots: (1) scatter p95_disp vs pbar; (2) coverage by quartiles 


In [27]:

import matplotlib.pyplot as plt
import numpy as np

# 1) Scatter with threshold line and rho annotation
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(dfj_haze["p95_disp_px"], dfj_haze["pbar"], s=16, alpha=0.6, edgecolors='k', linewidths=0.2)
ax.axhline(THR, linestyle='--', linewidth=1.0, label=f'Threshold T={THR:.3f}')
ax.set_xlabel("Per-frame displacement (95th percentile, pixels)")
ax.set_ylabel("Classifier probability (0–1)")
ax.set_title("DIC vs classifier score (joined haze frames)")
ax.legend(loc='lower right', frameon=True)
txt = f"Spearman ρ = {rho_p95:.2f}"
if not np.isnan(p_p95): txt += f" (p={p_p95:.3g})"
ax.text(0.02, 0.05, txt, transform=ax.transAxes)
fig.tight_layout()
scatter_path = FIGS_DIR / "dic_vs_pbar_scatter.png"
fig.savefig(scatter_path, dpi=200)
plt.close(fig)
print("Saved:", scatter_path)

# 2) Coverage by quartiles of p95_disp


# (A) Quartile coverage bar with counts; FutureWarning fixed via observed=True
q4 = pd.qcut(dfj_haze["p95_disp_px"], q=4)
def _short(iv): return f"{iv.left:.2f}–{iv.right:.2f}"
q4 = q4.cat.rename_categories([_short(iv) for iv in q4.cat.categories])

cov_by_bin = (
    dfj_haze.assign(bin=q4)
            .groupby("bin", observed=True)  # <- fixes the warning
            .agg(coverage=("pbar", lambda s: float((s >= THR).mean())),
                 n=("pbar","size"))
            .reset_index()
)

fig, ax = plt.subplots(figsize=(8,5))
ax.bar(cov_by_bin["bin"], cov_by_bin["coverage"])
for i,(c,n) in enumerate(zip(cov_by_bin["coverage"], cov_by_bin["n"])):
    ax.text(i, c+0.02, f"n={n}", ha="center", va="bottom", fontsize=9)
ax.set_ylim(0,1.05)
ax.set_xlabel("Quartile of per-frame displacement (p95, pixels)")
ax.set_ylabel("Coverage at T (fraction ≥ T)")
ax.set_title("Coverage by DIC displacement quartile")
fig.tight_layout()
fig.savefig(FIGS_DIR/"coverage_by_p95_quartile.png", dpi=200)
plt.close(fig)

# (B) Decile-binned median ± IQR of classifier score vs p95
q10 = pd.qcut(dfj_haze["p95_disp_px"], q=10, duplicates="drop")
q10 = q10.cat.rename_categories([_short(iv) for iv in q10.cat.categories])

summ10 = (
    dfj_haze.assign(bin=q10)
            .groupby("bin", observed=True)
            .agg(median=("pbar","median"),
                 q25=("pbar", lambda s: float(np.percentile(s,25))),
                 q75=("pbar", lambda s: float(np.percentile(s,75))),
                 cover=("pbar", lambda s: float((s>=THR).mean())),
                 n=("pbar","size"))
            .reset_index()
)

x = np.arange(len(summ10))
y = summ10["median"].to_numpy()
yerr = np.vstack([y - summ10["q25"].to_numpy(),
                  summ10["q75"].to_numpy() - y])

fig, ax = plt.subplots(figsize=(9,5))
ax.errorbar(x, y, yerr=yerr, fmt="o-", capsize=3, lw=1.5)
ax.axhline(THR, ls="--", lw=1.2, label=f"Threshold T={THR:.3f}")
for i,(yy,cov,n) in enumerate(zip(y, summ10["cover"], summ10["n"])):
    ax.text(i, yy+0.01, f"cov={cov:.2f}\n(n={n})", ha="center", va="bottom", fontsize=8)
ax.set_ylim(max(0.0, min(0.6, y.min()-0.05)), 1.02)
ax.set_xlim(-0.5, len(x)-0.5)
ax.set_xticks(x); ax.set_xticklabels(summ10["bin"], rotation=45, ha="right")
ax.set_xlabel("DIC displacement p95 (deciles, pixels)")
ax.set_ylabel("Classifier probability (median, IQR)")
ax.set_title("Classifier score vs DIC displacement (deciles)")
ax.legend(loc="lower left")
fig.tight_layout()
fig.savefig(FIGS_DIR/"pbar_vs_dic_deciles.png", dpi=200)
plt.close(fig)

print("Saved:",
      FIGS_DIR/"coverage_by_p95_quartile.png",
      FIGS_DIR/"pbar_vs_dic_deciles.png")


Saved: outputs/figs/dic_vs_pbar_scatter.png
Saved: outputs/figs/coverage_by_p95_quartile.png outputs/figs/pbar_vs_dic_deciles.png
